# Use BERTweet Representations with LogisticRegression Softmax Classifier

In [1]:
from collections import Counter
import os
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
import torch.nn as nn
from torch.utils.data import TensorDataset, Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from transformers import AutoModel, AutoTokenizer 


import dataset
import vsm
import sst

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# bertweet = AutoModel.from_pretrained("vinai/bertweet-large")
# tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large")

Downloading:   0%|          | 0.00/614 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
# bertweet_base = AutoModel.from_pretrained("vinai/bertweet-base", use_fast=False)
# tokenizer_base = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

AttributeError: 'NoneType' object has no attribute '__name__'

In [2]:
TWITTER = 2
TWITTER_AIRLINES = 3
TWITTER_APPLE = 4

In [3]:
twitter_train, twitter_validate, twitter_test =  dataset.dataset_reader(TWITTER_AIRLINES)
[twitter_train, twitter_validate, twitter_test] = list(map(lambda ds : dataset.prune_columns(TWITTER_AIRLINES, ds), [twitter_train, twitter_validate, twitter_test]))

In [4]:
# bert_weights_name = 'bert-base-cased'
# bert_tokenizer = BertTokenizer.from_pretrained(bert_weights_name)
# bert_model = BertModel.from_pretrained(bert_weights_name)
# model = BertForSequenceClassification.from_pretrained(bert_weights_name)
# Unique values of sentiment
twitter_sentiment_labels = twitter_train['sentiment'].unique()

In [5]:
twitter_train.size, twitter_validate.size, twitter_test.size

(58560, 7320, 7320)

In [6]:
%%time
bertweet_experiment1500 = sst.experiment(
    twitter_train[:1500], # 
    dataset.bert_tweet_phi,
    dataset.fit_softmax_classifier,
    assess_dataframes=[twitter_validate[:1000]],
    vectorize=False)

              precision    recall  f1-score   support

    negative      0.786     0.938     0.855       612
     neutral      0.639     0.454     0.531       207
    positive      0.724     0.492     0.586       181

    accuracy                          0.757      1000
   macro avg      0.716     0.628     0.657      1000
weighted avg      0.745     0.757     0.739      1000

CPU times: user 47min 7s, sys: 4.73 s, total: 47min 11s
Wall time: 11min 58s


In [7]:
%%time
bertweet_experiment3000 = sst.experiment(
    twitter_train[:3000], 
    dataset.bert_tweet_phi,
    dataset.fit_softmax_classifier,
    assess_dataframes=[twitter_validate[:1000]],
    vectorize=False)

              precision    recall  f1-score   support

    negative      0.805     0.922     0.859       612
     neutral      0.639     0.488     0.553       207
    positive      0.730     0.569     0.640       181

    accuracy                          0.768      1000
   macro avg      0.725     0.660     0.684      1000
weighted avg      0.757     0.768     0.756      1000

CPU times: user 1h 17min 17s, sys: 7.78 s, total: 1h 17min 25s
Wall time: 19min 38s


In [ ]:
%%time
bertweet_experiment6000 = sst.experiment(
    twitter_train[:6000], # 
    dataset.bert_tweet_phi,
    dataset.fit_softmax_classifier,
    assess_dataframes=[twitter_validate[:1500]],
    vectorize=False)

In [ ]:
%%time
bertweet_experiment_full = sst.experiment(
    twitter_train, # 
    dataset.bert_tweet_phi,
    dataset.fit_softmax_classifier,
    assess_dataframes=[twitter_validate],
    vectorize=False)

In [ ]:
bertweet_experiment_full.keys()

In [ ]:
bertweet_experiment_full['scores']

In [ ]:
bertweet_experiment_full['metric']

In [ ]:
bertweet_experiment_full['model']

----

# Test BERTweet trained on Tweets on test set

In [ ]:
import importlib
importlib.reload(sst)

In [ ]:
%%time
bertweet_test = sst.evaluate(
    bertweet_experiment_full['model'],
    bertweet_experiment_full['phi'],
    assess_dataframes=[twitter_test],
    vectorizer=bertweet_experiment_full['assess_datasets'][0]['vectorizer'],
    vectorize=False
)

In [ ]:
type(bertweet_test['predictions'][0])

In [ ]:
predictions_fname ='results/BERTweet_predictions_on_twitter_test_airline.csv'
df = bertweet_test['predictions'][0]
pd.DataFrame(df).to_csv(predictions_fname)

In [ ]:
encoding_fname ='results/BERTweet_encodings_on_twitter_test_airline.csv'
encoded_test = bertweet_test['assess_datasets'][0]
pd.DataFrame(encoded_test).to_csv(encoding_fname)

In [ ]:
predictions_df = pd.DataFrame(df)
predictions_df = predictions_df.set_index(twitter_test.index)
predictions_df

In [ ]:
twitter_test['BERTweet_sentiment'] = predictions_df

In [ ]:
twitter_test

In [ ]:
test_predictions_fname ='results/BERTweet_predictions_added_to_twitter_test_airline.csv'
twitter_test.to_csv(test_predictions_fname)

In [ ]:
correct = twitter_test[twitter_test['sentiment'] == twitter_test['BERTweet_sentiment']]

In [ ]:
correct

In [ ]:
incorrect = twitter_test[twitter_test['sentiment'] != twitter_test['BERTweet_sentiment']]

In [ ]:
incorrect

# Save Model

In [ ]:
import pickle
model_fname = 'models/BERTweet_twitter_airline.sav'
pickle.dump(bertweet_experiment_full, open(model_fname, 'wb'))